Create a scanimation (barrier-grid animation) from a set of images

Input: image images you want to animate or video to obtain images from
Output: one scanimation image (image images spliced together) that is viewed through a barrier of moving lines

1. Provide the paths for the set of images (or images grabed from a video) you want to animate, specify desired image parameters, and call functions to create the scanimation. Specify the desired object(s) to detect in each image. Adjust images to be the specified size and optionally align the desired object to ensure consistent placement throughout images. Specify the direction of animation (horizontal or vertical).

2. Read in the images. if object is specified, for each image detect the objects in the image and replace the original image with the detected image object overlayed over a white background.

3. Splice the images together into a single image.

4. Create a barrier of moving lines and view the spliced image in action as the lines move across the image.

### Utilities

In [23]:
# ---------------------------- Utilities ----------------------------

from pathlib import Path

def count_images(images_path):
    import os
    exts = {".png", ".jpg", ".jpeg", ".gif", ".bmp", ".tif", ".tiff", ".svg"}
    return sum(
        1 for f in os.listdir(images_path)
        if os.path.isfile(os.path.join(images_path, f))
        and os.path.splitext(f)[1].lower() in exts
    )

def _u2pt(units: str | None, dpi: float = 300.0) -> float:
    """
    Convert from `units` to PDF points (pt). 1 in = 72 pt.
    For 'px', conversion is DPI-aware: 1 px = 72/dpi pt.
    """
    u = (units or "in").strip().lower()
    if u in ("in", "inch", "inches"):  # inches
        return 72.0
    if u == "mm":                      # millimeters
        return 72.0 / 25.4
    if u == "cm":                      # centimeters
        return 72.0 / 2.54
    if u in ("pt", "pts", "point", "points"):  # points
        return 1.0
    if u in ("px", "pixel", "pixels"):         # pixels at given DPI
        if dpi <= 0:
            raise ValueError("dpi must be > 0 when converting from pixels.")
        return 72.0 / float(dpi)
    raise ValueError(f"Unsupported unit: {units!r}")

def units_to_pt(value: float, units: str | None, dpi: float = 300.0) -> float:
    return float(value) * _u2pt(units, dpi)

def pt_to_units(value_pt: float, units: str | None, dpi: float = 300.0) -> float:
    return float(value_pt) / _u2pt(units, dpi)
    
def units_to_px(value, units, dpi=300):
    """
    Convert a physical length 'value' in 'units' to pixels at 'dpi'.
    Supported units: in, mm, cm, pt, px
    """
    if value is None:
        return None
    u = (units or "").lower()
    v = float(value)
    if u in ("px","pixel","pixels"): return int(round(v))
    if u in ("in","inch","inches"):  return int(round(v * dpi))
    if u == "mm":                     return int(round((v / 25.4) * dpi))
    if u == "cm":                     return int(round((v / 2.54) * dpi))
    if u in ("pt","pts","point","points"): return int(round((v / 72.0) * dpi))
    raise ValueError(f"Unsupported units: {units!r}")

def measurements_to_px(image_width, image_height, barrier_width, slit_width,
                       image_units="in", barrier_units="mm", num_images=None, dpi=300):
    """
    Convert final physical geometry to pixel geometry once.
    Returns W_px, H_px, b_px, s_px (all ints).
    Rules:
      - if both barrier_width and slit_width given: check consistency (bar = (N-1)*slit)
      - else derive the missing one if num_images is provided
    """
    
    # Page size -> px
    W_px = units_to_px(image_width,  image_units)
    H_px = units_to_px(image_height, image_units)

    # Slit/bar -> px
    s_px = units_to_px(slit_width,  barrier_units) if slit_width  is not None else None
    b_px = units_to_px(barrier_width, barrier_units) if barrier_width is not None else None

    if num_images is None:
        # We need N to enforce b_px = (N-1)*s_px
        raise ValueError("measurements_to_px requires num_images to enforce integer geometry.")

    if s_px is not None:
        s_px = max(1, int(round(s_px)))
        b_px = (num_images - 1) * s_px
    elif b_px is not None:
        b_px = max(0, int(round(b_px)))
        s_px = max(1, int(round(b_px / float(num_images - 1))))
        b_px = (num_images - 1) * s_px
    else:
        raise ValueError("Provide slit_width or barrier_width.")

    return W_px, H_px, b_px, s_px

def _parse_color_to_rgb01(color):
    """
    Accepts '#RRGGBB' or '#RGB' or a 3-tuple/list of floats in [0,1].
    Returns (r,g,b) in 0..1.
    """
    if isinstance(color, (tuple, list)) and len(color) == 3:
        r, g, b = color
        return float(r), float(g), float(b)
    if isinstance(color, str):
        s = color.strip()
        if s.startswith("#"):
            s = s[1:]
            if len(s) == 3:
                r = int(s[0]*2, 16)
                g = int(s[1]*2, 16)
                b = int(s[2]*2, 16)
            elif len(s) == 6:
                r = int(s[0:2], 16)
                g = int(s[2:4], 16)
                b = int(s[4:6], 16)
            else:
                raise ValueError("Hex color must be #RGB or #RRGGBB")
            return (r/255.0, g/255.0, b/255.0)
    # default black if unknown
    return (0.0, 0.0, 0.0)

def _pdf_color_rg(r, g, b) -> str:
    """PDF fill color command in 0..1."""
    return f"{_fmt(r)} {_fmt(g)} {_fmt(b)} rg"

def save_flate_pdf(img_bgr, pdf_path, dpi):
    """
    Embed a BGR (or Gray) image as a Flate-compressed Image XObject
    into a minimal single-page PDF sized to print at 100% for the given DPI.
    """
    import cv2, zlib

    H, W = img_bgr.shape[:2]
    Wpt = (W / float(dpi)) * 72.0
    Hpt = (H / float(dpi)) * 72.0

    # Prepare raw bytes and color space
    if img_bgr.ndim == 2 or img_bgr.shape[2] == 1:
        # grayscale
        gray = img_bgr if img_bgr.ndim == 2 else img_bgr[:, :, 0]
        raw_bytes = gray.tobytes()
        color_space = "/DeviceGray"
        bpc = 8
    else:
        # BGR -> RGB
        rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        raw_bytes = rgb.tobytes()
        color_space = "/DeviceRGB"
        bpc = 8

    flate = zlib.compress(raw_bytes)

    header = "%PDF-1.4\n%\xE2\xE3\xCF\xD3\n"

    objs = []
    def obj(s: str) -> bytes:
        return s.encode("latin-1")

    # 1 Catalog
    objs.append("1 0 obj\n<< /Type /Catalog /Pages 2 0 R >>\nendobj\n")
    # 2 Pages
    objs.append("2 0 obj\n<< /Type /Pages /Count 1 /Kids [3 0 R] >>\nendobj\n")
    # 3 Page (references /Im1 and /Contents)
    objs.append(
        "3 0 obj\n"
        f"<< /Type /Page /Parent 2 0 R "
        f"/MediaBox [0 0 {Wpt:.6f} {Hpt:.6f}] "
        f"/Resources << /XObject << /Im1 4 0 R >> >> "
        f"/Contents 5 0 R >>\n"
        "endobj\n"
    )
    # 4 Image XObject (we’ll write the binary stream inline)
    img_dict = (
        "4 0 obj\n"
        f"<< /Type /XObject /Subtype /Image /Width {W} /Height {H} "
        f"/ColorSpace {color_space} /BitsPerComponent {bpc} /Filter /FlateDecode "
        f"/Length {len(flate)} >>\n"
        "stream\n"
    )
    img_end = "endstream\nendobj\n"

    # 5 Contents: draw image full-page
    # Use a Y-flip so top row of the buffer appears at top of the page.
    contents = f"q {Wpt:.6f} 0 0 {-Hpt:.6f} 0 {Hpt:.6f} cm /Im1 Do Q"
    objs.append(f"5 0 obj\n<< /Length {len(contents)} >>\nstream\n{contents}\nendstream\nendobj\n")

    # Write file and build xref
    with open(pdf_path, "wb") as f:
        xref = [0]  # object 0 placeholder
        offset = 0

        def w(b: bytes):
            nonlocal offset
            f.write(b); offset += len(b)

        w(obj(header))

        # obj 1
        xref.append(offset); w(obj(objs[0]))
        # obj 2
        xref.append(offset); w(obj(objs[1]))
        # obj 3
        xref.append(offset); w(obj(objs[2]))
        # obj 4 (image)
        xref.append(offset); w(obj(img_dict)); w(flate); w(obj("\n"+img_end))
        # obj 5 (contents)
        xref.append(offset); w(obj(objs[3]))

        startxref = offset
        # xref table
        lines = ["xref", "0 6", "0000000000 65535 f "]
        for off in xref[1:]:
            lines.append(f"{off:010d} 00000 n ")
        w(("\n".join(lines) + "\n").encode("latin-1"))

        # trailer
        w(f"trailer\n<< /Size 6 /Root 1 0 R >>\nstartxref\n{startxref}\n%%EOF\n".encode("latin-1"))

def save_png_in_svg(bgr_img, svg_path, width_in, height_in, pixel_w, pixel_h):
    """Embed PNG data URI in an SVG with physical size in inches and viewBox in pixels."""
    from PIL import Image
    from io import BytesIO
    import base64
    import cv2
    
    # Convert BGR -> RGB for PNG encoding
    pil_img = Image.fromarray(cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB))
    buf = BytesIO()
    pil_img.save(buf, format="PNG")
    b64_png = base64.b64encode(buf.getvalue()).decode("ascii")
    svg = f'''<?xml version="1.0" encoding="UTF-8"?>
<svg xmlns="http://www.w3.org/2000/svg"
     width="{width_in:.6f}in" height="{height_in:.6f}in"
     viewBox="0 0 {pixel_w} {pixel_h}">
  <image href="data:image/png;base64,{b64_png}" x="0" y="0" width="{pixel_w}" height="{pixel_h}" />
</svg>'''
    with open(svg_path, "w", encoding="utf-8") as f:
        f.write(svg)

def _resolve_out(out_name: str, base_dir: str) -> Path:
        p = Path(out_name)
        # If caller provided just a filename (no parent dir), place it in output_path
        if not p.is_absolute() and p.parent == Path('.'):
            return Path(base_dir) / p.name
        return p

def sort_key(filename): # Numeric sort
    import re
    m = re.search(r'(\d+)', filename)
    return int(m.group(1)) if m else 0

def _fmt(n):
    return f"{n:.4f}".rstrip('0').rstrip('.') if isinstance(n, float) else str(n)

def _svgu(val, units):  # value + unit for SVG attributes
    return f"{val:.6f}{units}"

In [24]:
def get_images_from_video(video_path, num_images=6, start_time=None, end_time=None):    
    # Extract the specified number of images from a video file and save the images to a new folder 
    
    import cv2
    import os
    import numpy as np

    # File Handling
    output_path = ''
    # if video_path is a directory, use the first video file in it.
    if os.path.isdir(video_path):
        video_files = [f for f in os.listdir(video_path) if f.endswith(('.mp4', '.avi', '.mov'))]
        if not video_files:
            raise ValueError("No video files found in the specified directory.")
        video_path = os.path.join(video_path, video_files[0])
        output_path = os.path.join(os.path.dirname(video_path), 'images')
    # if video_path is a file, use it directly.    
    elif os.path.isfile(video_path):
        output_path = os.path.join(os.path.dirname(video_path), 'images')
    else:
        raise ValueError("The provided video path is neither a directory nor a valid file.")
    # Create the output directory if it does not exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Load video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video file: {video_path}")
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_images = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_sec = total_images / fps
    
    # Convert start and end times to seconds
    start_sec = start_time if start_time is not None else 0
    end_sec = end_time if end_time is not None else duration_sec

    if start_sec < 0 or end_sec > duration_sec or start_sec >= end_sec:
        raise ValueError("Invalid start_time or end_time")
    
    start_frame = int(start_sec * fps)
    end_frame = int(end_sec * fps)

    # Determine frame indices to extract
    frame_indices = np.linspace(start_frame, end_frame - 1, num_images, dtype=int)
    
    saved_count = 0
    for i in range(total_images):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            filename = os.path.join(output_path, f"frame_{saved_count:03d}.png")
            cv2.imwrite(filename, frame)
            saved_count += 1
        if saved_count >= num_images:
            break

    cap.release()
    print(f"Saved {saved_count} images to '{output_path}'.")
    return output_path


In [25]:
def calculate_ideal_fit(
    images_path: str,
    image_width: float | None,
    image_height: float | None,
    image_units: str,
    barrier_width: float | None,
    slit_width: float | None,
    barrier_units: str,
    num_images: int | None,
    image_size_strict: bool = True,
    barrier_width_strict: bool = False,
    dpi: int = 300,
    suggest_target_slit_mm: float = 1.0,
    periods_hint_range: tuple[int, int] = (60, 240),  # used only when deriving from width
    tol_rel: float = 0.02,  # 2% tolerance for “close enough”
):
    """
    Compute harmonized image size + barrier geometry so periods fit exactly.
    Returns: (image_width, image_height, barrier_width, slit_width) in the original requested units.

    Rules:
      - barrier_width = (num_images - 1) * slit_width
      - period = num_images * slit_width

    Geometry first (derive bw/slit/N from what's provided):
      1) If barrier_width and slit_width are given:
           - Check that provided num_images (if given) matches this geometry.
             If not, raise ValueError and suggest an alternative num_images.
           - If num_images is None, infer it from bw/slit.
      2) elif barrier_width and num_images are given:
           - Compute slit_width = barrier_width / (num_images - 1).
      3) elif slit_width and num_images are given:
           - Compute barrier_width = (num_images - 1) * slit_width.
      4) elif only num_images is given:
           - Choose slit/bar so an integer number of periods fits across image width,
             targeting ~suggest_target_slit_mm if possible.

    Adjustments (fit to image width / preserve geometry):
      - image_size_strict:
          True  => keep image size as given.
          False => adjust image to be as close as possible to the specified image size while having exactly an integer number of periods. If this is False, dont't adjust the barrier and slit widths.
      - barrier_width_strict:
          True  => keep barrier_width exact; enforce slit = barrier/(N-1). If image width
                   is not a multiple of period and image_size_strict is True, the last period
                   will be partial (i.e., effectively cropped).
          False => adjust slit (and thus barrier via bw=(N-1)*slit) *slightly* so that only
                   full periods fit across the existing image width (unless image_size_strict=False,
                   in which case we may instead adjust image width).

    Notes:
      - Uses the first readable image in images_path to infer image size if needed.
      - Works internally in points; returns values in the original units requested.
    """
    import os
    import cv2

    # --- Helper: first image size (pixels) ---
    def _first_image_size_px(folder: str):
        exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"}
        for f in sorted(os.listdir(folder)):
            fp = os.path.join(folder, f)
            if os.path.isfile(fp) and os.path.splitext(f)[1].lower() in exts:
                img = cv2.imread(fp)
                if img is not None:
                    h, w = img.shape[:2]
                    return w, h
        raise RuntimeError(f"No readable raster images found in {folder}")

    # --- Derive image size if missing ---
    w0_px, h0_px = _first_image_size_px(images_path)
    if image_width is None:
        image_width = (w0_px / dpi)  # inches
        image_width = pt_to_units(image_width * 72.0, image_units)
    if image_height is None:
        image_height = (h0_px / dpi)  # inches
        image_height = pt_to_units(image_height * 72.0, image_units)

    imgW_pt = units_to_pt(image_width, image_units)
    imgH_pt = units_to_pt(image_height, image_units)

    # --- Geometry first: determine N, slit_pt, bar_pt ---
    N = num_images if num_images is not None else None
    slit_pt = None
    bar_pt = None

    def _close(a: float, b: float, rel: float) -> bool:
        denom = max(abs(a), abs(b), 1e-9)
        return abs(a - b) <= rel * denom

    if barrier_width is not None and slit_width is not None:
        # Case 1: both given
        bar_pt = units_to_pt(barrier_width, barrier_units)
        slit_pt = units_to_pt(slit_width, barrier_units)
        if slit_pt <= 0 or bar_pt <= 0:
            raise ValueError("barrier_width and slit_width must be positive.")
        implied_N = (bar_pt / slit_pt) + 1.0
        N_from_bw_slit = int(round(implied_N))
        if not _close(implied_N, N_from_bw_slit, tol_rel):
            raise ValueError(
                f"The ratio barrier/slit = {bar_pt/slit_pt:.4f} does not yield an integer N-1.\n"
                f"Suggested num_images ≈ {implied_N:.3f} → try N = {N_from_bw_slit}."
            )
        if N is None:
            N = N_from_bw_slit
        else:
            if N < 2:
                raise ValueError("num_images must be ≥ 2.")
            if N != N_from_bw_slit:
                # “ensure the given num_images would fit this geometry as closely as possible” → it doesn’t.
                raise ValueError(
                    f"Given num_images={N} conflicts with barrier/slit geometry (implies N={N_from_bw_slit}).\n"
                    f"Suggestion: use num_images={N_from_bw_slit} and manually pick the images you want to keep or change the barrier/slit geometry to fit your number of images (slit_width = barrier_width / (N-1))."
                )

    elif barrier_width is not None and N is not None:
        # Case 2: bw + N
        if N < 2:
            raise ValueError("num_images must be ≥ 2.")
        bar_pt = units_to_pt(barrier_width, barrier_units)
        slit_pt = bar_pt / (N - 1)

    elif slit_width is not None and N is not None:
        # Case 3: slit + N
        if N < 2:
            raise ValueError("num_images must be ≥ 2.")
        slit_pt = units_to_pt(slit_width, barrier_units)
        bar_pt = (N - 1) * slit_pt

    elif N is not None:
        # Case 4: only N given → pick slit so an integer number of periods fits img width
        if N < 2:
            raise ValueError("num_images must be ≥ 2.")
        target_slit_pt = units_to_pt(suggest_target_slit_mm, "mm")
        # Try to choose an integer number of periods (k) so slit ≈ target
        # slit = imgW_pt / (k * N)  => k = imgW_pt / (N * slit)
        k_float = imgW_pt / (N * target_slit_pt)
        candidate_ks = []
        k0 = int(round(k_float))
        lo, hi = periods_hint_range
        for k in {max(1, k0 - 2), max(1, k0 - 1), max(1, k0), k0 + 1, k0 + 2, lo, hi}:
            if lo <= k <= hi:
                candidate_ks.append(k)
        candidate_ks = sorted(set(candidate_ks))
        best = None
        for k in candidate_ks:
            candidate_slit = imgW_pt / (k * N)
            # Prefer slit close to target, but keep a practical band (0.3–3.0 mm)
            mm = pt_to_units(candidate_slit, "mm")
            score = abs(mm - suggest_target_slit_mm) + (0.0 if 0.3 <= mm <= 3.0 else 100.0)
            if best is None or score < best[0]:
                best = (score, candidate_slit, k)
        slit_pt = best[1]
        bar_pt = (N - 1) * slit_pt

    else:
        raise ValueError("Insufficient inputs. Provide at least num_images, or a combination that determines it (e.g., barrier+slit).")

    # Enforce the consistency constraint explicitly
    bar_pt = (N - 1) * slit_pt
    period_pt = N * slit_pt

    # --- Adjustments phase ---
    if barrier_width_strict:
        # Keep barrier exact; slit follows exactly from it
        # (Recompute slit to ensure consistency; don't adjust to fit width)
        slit_pt = bar_pt / (N - 1)
        period_pt = N * slit_pt
        if not image_size_strict:
            # Adjust image width to exact multiple of period
            n_periods = max(1, round(imgW_pt / period_pt))
            imgW_pt = n_periods * period_pt
        # else: leave image width as-is (may end with a partial period)
    else:
        # Adjust slit (and thus bar) *slightly* so only full periods fit the (possibly fixed) image width
        if image_size_strict:
            # Fit to current image width by nudging slit
            n_periods = max(1, round(imgW_pt / period_pt))
            slit_pt = imgW_pt / (n_periods * N)
            bar_pt = (N - 1) * slit_pt
            period_pt = N * slit_pt
        else:
            # Easier: snap image width to a multiple of the current period
            n_periods = max(1, round(imgW_pt / period_pt))
            imgW_pt = n_periods * period_pt
            # keep slit/bar as is (already consistent)

    # --- Return in requested units ---
    out_image_width = pt_to_units(imgW_pt, image_units)
    out_image_height = pt_to_units(imgH_pt, image_units)
    out_barrier_width = pt_to_units(bar_pt, barrier_units)
    out_slit_width = pt_to_units(slit_pt, barrier_units)

    # Diagnostics
    print("=== Ideal Fit ===")
    print(f"N (num_images): {N}")
    print(f"Image size: {out_image_width:.4f}{image_units} × {out_image_height:.4f}{image_units}")
    print(f"Slit width: {out_slit_width:.4f}{barrier_units}")
    print(f"Barrier width (opaque): {out_barrier_width:.4f}{barrier_units}")
    print(f"Period (= N*slit): {pt_to_units(period_pt, barrier_units):.4f}{barrier_units}")
    approx_periods = (units_to_pt(out_image_width, image_units)) / (N * units_to_pt(out_slit_width, barrier_units))
    print(f"Periods across width: ~{round(approx_periods)}")
    return out_image_width, out_image_height, out_barrier_width, out_slit_width

# Usage
#images_path = '/Users/macadmin/Desktop/Sara_Laptop/CodingProjects/Scanimation/seal/images'
#num_images = count_images(images_path)
#image_width, image_height, barrier_width, slit_width = calculate_ideal_fit(images_path, image_width=8.0, image_height=11.5, image_units='in', barrier_width=4.0, slit_width=1.0, barrier_units='mm', num_images=num_images, image_size_strict=True, barrier_width_strict=False, dpi=300, suggest_target_slit_mm=1.0, periods_hint_range=(60, 240), tol_rel=0.02)


### Create Barrier

In [26]:
def create_scanimation_barrier(
    output_path: str,             
    barrier_width: float,
    slit_width: float,
    image_width: float,
    image_height: float,
    image_units: str = "mm",      # canvas/page units for image_width/height
    barrier_units: str = "mm",    # units for barrier/slit widths
    W_px: int = None,
    H_px: int = None,
    b_px: int = None,
    s_px: int = None,
    horizontal_motion: bool = True,
    color = "#000000",            # bar color
    svg_out: str = "barrier.svg",
    pdf_out: str = "barrier.pdf",
    png_out: str = "barrier.png",
    dpi: int = 300
):
    """
    Create a scanimation barrier (bar-and-slit barrier) and export both SVG and PDF.

    Parameters
    ----------
    output_path : str
        Directory where output files are saved (created if missing).
    barrier_width : float
        Width of each opaque bar in `barrier_units`.
    slit_width : float
        Width of each transparent slit in `barrier_units`.
    image_width, image_height : float
        Overall canvas size in `image_units`.
    image_units : str
        Units for image size. One of: 'mm', 'cm', 'in', 'pt', 'px' (px assumed 72dpi).
    barrier_units : str
        Units for barrier/slit widths. Same supported set as `image_units`.
    horizontal_motion : bool
        If True, bars are vertical (slits vertical) -> suitable for horizontal animation.
        If False, bars are horizontal -> suitable for vertical animation.
    color : str | tuple
        Bar color as '#RRGGBB'/'#RGB', or (r,g,b) in [0,1] or [0,255].
    svg_out : str
        Filename or path for SVG. If bare filename, saved inside `output_path`.
    pdf_out : str
        Filename or path for PDF. If bare filename, saved inside `output_path`.

    Returns
    -------
    (svg_path_str, pdf_path_str) : tuple[str, str]
        Absolute or relative paths actually written.
    """
    from pathlib import Path
    import os
    import re
    import numpy as np
    from PIL import Image

    # Validate inputs
    if barrier_width <= 0 or slit_width < 0:
        raise ValueError("barrier_width must be > 0 and slit_width >= 0")
    if image_width <= 0 or image_height <= 0:
        raise ValueError("image_width and image_height must be > 0")

    # Prepare output dirs
    svg_path = _resolve_out(svg_out, output_path)
    pdf_path = _resolve_out(pdf_out, output_path)
    png_path = _resolve_out(png_out, output_path)
    svg_path.parent.mkdir(parents=True, exist_ok=True)
    pdf_path.parent.mkdir(parents=True, exist_ok=True)
    png_path.parent.mkdir(parents=True, exist_ok=True)

    # --- Unit conversions ---

    # Physical sizes in points (PDF space)
    Wpt  = units_to_pt(image_width, image_units)
    Hpt  = units_to_pt(image_height, image_units)
    bwpt = units_to_pt(barrier_width, barrier_units)
    swpt = units_to_pt(slit_width, barrier_units)
    period_pt = bwpt + swpt

    r, g, b = _parse_color_to_rgb01(color)

    # ---------- SVG ----------
    # SVG canvas uses image_units; convert barrier widths to same coordinate system
    U_img = _u2pt(image_units)
    U_bar = _u2pt(barrier_units)
    scale = U_bar / U_img  # converts barrier units to image-unit coords
    svg_lines = ['<?xml version="1.0" encoding="UTF-8" standalone="no"?>']
    svg_lines.append(
        f'<svg xmlns="http://www.w3.org/2000/svg" '
        f'width="{_fmt(image_width)}{image_units}" height="{_fmt(image_height)}{image_units}" '
        f'viewBox="0 0 {_fmt(image_width)} {_fmt(image_height)}">'
    )
    svg_lines.append("<title>Scanimation Barrier</title>")
    bar_fill = f"rgb({int(r*255)},{int(g*255)},{int(b*255)})"
    svg_lines.append(f'<g fill="{bar_fill}" stroke="none">')

    if horizontal_motion:
        # vertical bars; slits start at left
        x = slit_width * scale
        while x < image_width - 1e-9:
            w = min(barrier_width * scale, image_width - x)
            if w > 0:
                svg_lines.append(
                    f'<rect x="{_fmt(x)}" y="0" width="{_fmt(w)}" height="{_fmt(image_height)}"/>'
                )
            x += (barrier_width + slit_width) * scale
    else:
        # horizontal bars; slits start at top
        y = slit_width * scale
        while y < image_height - 1e-9:
            h = min(barrier_width * scale, image_height - y)
            if h > 0:
                svg_lines.append(
                    f'<rect x="0" y="{_fmt(y)}" width="{_fmt(image_width)}" height="{_fmt(h)}"/>'
                )
            y += (barrier_width + slit_width) * scale

    svg_lines.append("</g></svg>")
    with open(str(svg_path), "w", encoding="utf-8") as f:
        f.write("\n".join(svg_lines))

    # ---------- PDF ----------
    # Build a minimal PDF with filled rectangles for the bars.
    content = []
    content.append(_pdf_color_rg(r, g, b))  # set fill color

    if horizontal_motion:
        x = swpt
        while x < Wpt - 1e-9:
            w = min(bwpt, Wpt - x)
            if w > 0:
                content.append(f"{_fmt(x)} 0 {_fmt(w)} {_fmt(Hpt)} re f")
            x += period_pt
    else:
        y = swpt
        while y < Hpt - 1e-9:
            h = min(bwpt, Hpt - y)
            if h > 0:
                content.append(f"0 {_fmt(y)} {_fmt(Wpt)} {_fmt(h)} re f")
            y += period_pt

    stream = "\n".join(content).encode("ascii")

    header = "%PDF-1.4\n%\xE2\xE3\xCF\xD3\n"
    objs, xref = [], []
    byte_count = len(header.encode("latin-1"))

    def add_obj(sobj: str):
        nonlocal byte_count
        xref.append(byte_count)
        objs.append(sobj)
        byte_count += len(sobj.encode("latin-1"))

    # Objects: Catalog, Pages, Page, Contents
    add_obj("1 0 obj\n<< /Type /Catalog /Pages 2 0 R >>\nendobj\n")
    add_obj("2 0 obj\n<< /Type /Pages /Count 1 /Kids [3 0 R] >>\nendobj\n")
    add_obj(
        f"3 0 obj\n<< /Type /Page /Parent 2 0 R "
        f"/MediaBox [0 0 {_fmt(Wpt)} {_fmt(Hpt)}] "
        f"/Contents 4 0 R >>\nendobj\n"
    )
    add_obj(
        f"4 0 obj\n<< /Length {len(stream)} >>\nstream\n{stream.decode('ascii')}\nendstream\nendobj\n"
    )

    xref_pos = byte_count
    xref_table = (
        "xref\n0 " + str(len(xref) + 1) + "\n"
        "0000000000 65535 f \n" +
        "".join(f"{off:010d} 00000 n \n" for off in xref)
    )
    trailer = f"trailer\n<< /Size {len(xref)+1} /Root 1 0 R >>\nstartxref\n{xref_pos}\n%%EOF\n"

    with open(str(pdf_path), "wb") as f:
        f.write(header.encode("latin-1"))
        for s_obj in objs:
            f.write(s_obj.encode("latin-1"))
        f.write(xref_table.encode("latin-1"))
        f.write(trailer.encode("latin-1"))

    # ---------- PNG rasterization with transparent slits ----------
    # Compute raster dimensions
    if not (W_px and H_px and b_px is not None and s_px is not None):
        # If not provided, you could compute here — but in this pipeline we REQUIRE them
        #W_px = int(round(units_to_px(image_width,  image_units, dpi)))
        #H_px = int(round(units_to_px(image_height, image_units, dpi)))
        #s_px = max(1, int(round(units_to_px(slit_width,   barrier_units, dpi))))
        #b_px = max(0, int(round(units_to_px(barrier_width, barrier_units, dpi))))
        raise ValueError("W_px, H_px, b_px, s_px must be provided for pixel-accurate PNG barrier.")
    period_px = s_px + b_px

    # RGBA canvas: slits transparent, bars opaque color
    rgba = np.zeros((H_px, W_px, 4), dtype=np.uint8)
    R, G, B = int(r*255), int(g*255), int(b*255)
    if horizontal_motion:
        x = s_px
        while x < W_px:
            w = min(b_px, W_px - x)
            if w > 0:
                rgba[:, x:x+w, 0] = R
                rgba[:, x:x+w, 1] = G
                rgba[:, x:x+w, 2] = B
                rgba[:, x:x+w, 3] = 255
            x += period_px
    else:
        y = s_px
        while y < H_px:
            h = min(b_px, H_px - y)
            if h > 0:
                rgba[y:y+h, :, 0] = R
                rgba[y:y+h, :, 1] = G
                rgba[y:y+h, :, 2] = B
                rgba[y:y+h, :, 3] = 255
            y += period_px

    # Save with DPI metadata
    img = Image.fromarray(rgba, mode="RGBA")
    img.save(str(png_path), dpi=(dpi, dpi))
    
    return str(svg_path), str(pdf_path), str(png_path)


# -------------------- Usage --------------------
"""
images_path = '/Users/macadmin/Desktop/Sara_Laptop/CodingProjects/Scanimation/seal'
output_path = f'{images_path}/scanimated'

create_scanimation_barrier(
    output_path,
    barrier_width=4, slit_width=1,   
    image_width=11, image_height=8.5,  
    image_units="in",
    barrier_units="mm",
    horizontal_motion=True,
    color="#000000",
    svg_out="barrier.svg",
    pdf_out="barrier.pdf",
)
"""

'\nimages_path = \'/Users/macadmin/Desktop/Sara_Laptop/CodingProjects/Scanimation/seal\'\noutput_path = f\'{images_path}/scanimated\'\n\ncreate_scanimation_barrier(\n    output_path,\n    barrier_width=4, slit_width=1,   \n    image_width=11, image_height=8.5,  \n    image_units="in",\n    barrier_units="mm",\n    horizontal_motion=True,\n    color="#000000",\n    svg_out="barrier.svg",\n    pdf_out="barrier.pdf",\n)\n'

### Process Images

In [27]:
def identify_objects(images_path, objects_to_detect=None):
    # For each image in the images_path, identify objects, clear the background, and save the new images in a new folder
    from ultralytics import YOLO
    import cv2
    import numpy as np
    import os

    processing_path = os.path.join(images_path, 'identified')
    os.makedirs(processing_path, exist_ok=True)

    model = YOLO("yolov8n-seg.pt")  # Segmentation model for masking

    for fname in os.listdir(images_path):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        img_path = os.path.join(images_path, fname)
        image = cv2.imread(img_path)

        # Run inference
        results = model(img_path)[0]

        mask_combined = np.zeros(image.shape[:2], dtype=np.uint8)

        # If objects_to_detect is specified, filter results
        for i, cls in enumerate(results.names.values()):
            if objects_to_detect and cls not in objects_to_detect:
                continue

        # If no objects specified, just use all masks
        for m in results.masks.data:
            m = m.cpu().numpy()
            m = (m * 255).astype(np.uint8)
            # Resize m to match original image dimensions
            m_resized = cv2.resize(m, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
            # Combine masks
            mask_combined = cv2.bitwise_or(mask_combined, m_resized)

        # Apply mask
        masked = cv2.bitwise_and(image, image, mask=mask_combined)
        white_bg = np.full_like(image, 255)
        inv_mask = cv2.bitwise_not(mask_combined)
        cleared = cv2.bitwise_and(white_bg, white_bg, mask=inv_mask)
        result = cv2.add(masked, cleared)

        out_path = os.path.join(processing_path, fname)
        cv2.imwrite(out_path, result)

    print(f"Saved processed images to {processing_path}")

    return processing_path

In [28]:
def monocolorize_images(images_path, color='black', invert=False, out_dir=None):
    """
    Take images in `images_path` that already have an object mask baked-in (e.g., from identify_objects)
    and produce monochrome images where the object is filled with `color` and background is white.
    If `invert=True`, swap object/background colors.

    Saves outputs to `out_dir` (or in-place if None). Returns the output path.
    """
    import os, cv2, numpy as np

    output_images_path = out_dir or images_path
    os.makedirs(output_images_path, exist_ok=True)

    # Resolve target color -> BGR
    def parse_color(c):
        if isinstance(c, (tuple, list)) and len(c) == 3:
            r, g, b = c
            return (int(b), int(g), int(r))  # to BGR
        c = (c or 'black').strip().lower()
        named = {
            'black': (0, 0, 0),
            'white': (255, 255, 255),
            'red':   (0, 0, 255),
            'green': (0, 255, 0),
            'blue':  (255, 0, 0),
        }
        if c in named:
            return named[c]
        if c.startswith('#'):
            s = c[1:]
            if len(s) == 3: s = ''.join(ch*2 for ch in s)
            r = int(s[0:2], 16); g = int(s[2:4], 16); b = int(s[4:6], 16)
            return (b, g, r)
        return (0, 0, 0)
    target_bgr = parse_color(color)

    # Process each image
    image_files = [f for f in os.listdir(images_path)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for fname in image_files:
        img_path = os.path.join(images_path, fname)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"Warning: failed to read {fname}; skipping.")
            continue

        H, W = img.shape[:2]

        # Build a mask by detecting *non-white* pixels (since identify_objects wrote white background).
        # If you have an explicit mask available, load/use it instead.
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Treat near-white as background
        mask = (gray < 250).astype(np.uint8) * 255  # 0/255 uint8
        # Ensure mask is 1-channel uint8
        mask = mask.astype(np.uint8)

        # Foreground (object) filled with target color
        mono = np.full_like(img, target_bgr, dtype=np.uint8)
        fg = cv2.bitwise_and(mono, mono, mask=mask)

        # Background white
        inv = cv2.bitwise_not(mask)
        bg = np.full_like(img, 255, dtype=np.uint8)
        bg = cv2.bitwise_and(bg, bg, mask=inv)

        out = cv2.add(fg, bg)

        if invert:
            out = 255 - out  # simple invert of colors

        cv2.imwrite(os.path.join(output_images_path, fname), out)

    print(f"Monocolorized {len(image_files)} images → {output_images_path}")
    return output_images_path

In [29]:
def resize_images(images_path, image_width=None, image_height=None, units='px',
                  dpi=300, preserve_aspect_ratio=True, crop_to_fit=True):
    import os
    import cv2
    import numpy as np
    from PIL import Image

    # File Handling
    processing_path = os.path.join(images_path, 'resized')
    os.makedirs(processing_path, exist_ok=True)

    image_files = [f for f in os.listdir(images_path)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not image_files:
        raise ValueError("No image files found in the provided directory.")

    def to_pixels(value, units, dpi):
        if value is None:
            return None
        if units == 'px':
            return int(value)
        elif units == 'in':
            return int(round(value * dpi))
        elif units == 'cm':
            return int(round((value / 2.54) * dpi))
        elif units == 'mm':
            return int(round((value / 25.4) * dpi))
        else:
            raise ValueError("Unsupported unit. Use 'px', 'in', 'cm', or 'mm'.")

    # Use the first image as reference
    first_image_path = os.path.join(images_path, image_files[0])
    first_image = cv2.imread(first_image_path)
    if first_image is None:
        raise ValueError("Failed to load the first image for reference sizing.")
    ref_h, ref_w = first_image.shape[:2]

    width_px  = to_pixels(image_width,  units, dpi) if image_width  is not None else None
    height_px = to_pixels(image_height, units, dpi) if image_height is not None else None

    for fname in image_files:
        img_path = os.path.join(images_path, fname)
        image = cv2.imread(img_path)
        if image is None:
            print(f"Warning: could not load image {fname}. Skipping.")
            continue

        h_orig, w_orig = image.shape[:2]
        aspect_ratio = w_orig / h_orig

        # --- Resizing logic (same as before) ---
        if width_px is None and height_px is None:
            new_w, new_h = ref_w, ref_h
            resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)

        elif preserve_aspect_ratio and not crop_to_fit:
            if width_px and height_px:
                scale = min(width_px / w_orig, height_px / h_orig)
                new_w = max(1, int(round(w_orig * scale)))
                new_h = max(1, int(round(h_orig * scale)))
            elif width_px:
                new_w = width_px
                new_h = max(1, int(round(width_px / aspect_ratio)))
            elif height_px:
                new_h = height_px
                new_w = max(1, int(round(height_px * aspect_ratio)))
            else:
                new_w, new_h = w_orig, h_orig
            resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)

        elif not preserve_aspect_ratio and not crop_to_fit:
            new_w = width_px if width_px else w_orig
            new_h = height_px if height_px else h_orig
            resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)

        elif crop_to_fit and width_px and height_px:
            target_aspect = width_px / height_px
            if aspect_ratio > target_aspect:
                scale = height_px / h_orig
                scaled_w = max(1, int(round(w_orig * scale)))
                temp = cv2.resize(image, (scaled_w, height_px), interpolation=cv2.INTER_AREA)
                x_start = max(0, (scaled_w - width_px) // 2)
                resized = temp[:, x_start:x_start + width_px]
            else:
                scale = width_px / w_orig
                scaled_h = max(1, int(round(h_orig * scale)))
                temp = cv2.resize(image, (width_px, scaled_h), interpolation=cv2.INTER_AREA)
                y_start = max(0, (scaled_h - height_px) // 2)
                resized = temp[y_start:y_start + height_px, :]

            resized = cv2.resize(resized, (width_px, height_px), interpolation=cv2.INTER_AREA)

        else:
            raise ValueError("To crop to fit, both width and height must be provided; "
                             "or set crop_to_fit=False for single-dimension resizing.")

        # --- Save with DPI metadata ---
        out_path = os.path.join(processing_path, fname)
        rgb = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb)
        pil_img.save(out_path, dpi=(dpi, dpi))

    mode = (
        "cropped to fit box" if (crop_to_fit and width_px and height_px) else
        "preserved aspect ratio" if (preserve_aspect_ratio and not crop_to_fit) else
        "forced exact size" if (not preserve_aspect_ratio and not crop_to_fit and (width_px or height_px)) else
        "matched first image"
    )
    size_note = (f"{image_width or ref_w} x {image_height or ref_h} {units}"
                 if (image_width or image_height) else f"{ref_w} x {ref_h} (matched first image)")
    print(f"Resized {len(image_files)} images to {size_note} at {dpi} DPI using mode: {mode}")

    return processing_path

### Make interlaced image

In [30]:
def interlace_images(images_path, output_path,
                     W_px: int, H_px: int, b_px: int, s_px: int,
                     horizontal_motion: bool = True,
                     dpi: int = 300,
                     out_name: str = "interlaced",
                     ):
    """
    Build the interlaced image using exact pixel geometry.

    Saves:
      - <out_name>.png  (always)
      - <out_name>.pdf  (lossless)
      - <out_name>.svg  (true-to-size)
    """
    import os, re, cv2, numpy as np
    from PIL import Image
    from io import BytesIO
    import base64

    os.makedirs(output_path, exist_ok=True)

    # ---- load frames ----
    files = sorted(
        [f for f in os.listdir(images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))],
        key=sort_key
    )
    if len(files) < 2:
        raise ValueError("Need >= 2 images to interlace.")

    imgs = []
    for f in files:
        im = cv2.imread(os.path.join(images_path, f), cv2.IMREAD_COLOR)
        if im is None:
            continue
        if im.shape[1] != W_px or im.shape[0] != H_px:
            raise ValueError(
                f"{f} is {im.shape[1]}x{im.shape[0]} px, expected {W_px}x{H_px} px. "
                "Run resize_images with units='px' first."
            )
        imgs.append(im)

    N = len(imgs)

    # ---- sanity: enforce b_px = (N-1)*s_px ----
    exp_b = (N - 1) * s_px
    if b_px != exp_b:
        raise ValueError(f"b_px={b_px} but expected (N-1)*s_px={(N-1)}*{s_px}={exp_b}")

    # ---- interlace ----
    interlaced = np.zeros((H_px, W_px, 3), dtype=np.uint8)

    if horizontal_motion:
        # vertical strips; advance along X
        if W_px % s_px != 0:
            print(f"[warn] width {W_px} not divisible by s_px {s_px} (remainder {W_px % s_px})")
        x0, k = 0, 0
        while x0 < W_px:
            x1 = min(x0 + s_px, W_px)
            interlaced[:, x0:x1, :] = imgs[k % N][:, x0:x1, :]
            x0, k = x1, k + 1
    else:
        # horizontal strips; advance along Y
        if H_px % s_px != 0:
            print(f"[warn] height {H_px} not divisible by s_px {s_px} (remainder {H_px % s_px})")
        y0, k = 0, 0
        while y0 < H_px:
            y1 = min(y0 + s_px, H_px)
            interlaced[y0:y1, :, :] = imgs[k % N][y0:y1, :, :]
            y0, k = y1, k + 1

    # ---- save PNG ----
    png_path = os.path.join(output_path, f"{out_name}.png")
    cv2.imwrite(png_path, interlaced)

    # ---- Save PDF (lossless Flate) ----
    pdf_path = os.path.join(output_path, f"{out_name}.pdf")
    save_flate_pdf(interlaced, pdf_path, dpi=dpi)

    # ---- Save SVG (true physical size) ----
    width_in  = W_px / float(dpi)
    height_in = H_px / float(dpi)
    svg_path = os.path.join(output_path, f"{out_name}.svg")
    save_png_in_svg(interlaced, svg_path, width_in, height_in, W_px, H_px)

    print(f"[interlace_images] N={N}, s_px={s_px}, b_px={b_px}, size={W_px}x{H_px}, saved: {png_path}")
    return

### View Scanimation

In [31]:
def view_scanimation(output_path,
                     W_px: int, H_px: int, b_px: int, s_px: int,
                     horizontal_motion=True, reverse_motion=False,
                     fps=12, cycles=2,
                     gif_name="preview.gif", mov_name="preview.mov"):
    import os, cv2, numpy as np, imageio.v2 as imageio

    inter_path = os.path.join(output_path, "interlaced.png")
    bar_path   = os.path.join(output_path, "barrier.png")
    inter = cv2.imread(inter_path, cv2.IMREAD_COLOR)
    bar   = cv2.imread(bar_path, cv2.IMREAD_UNCHANGED)  # keep alpha
    if inter is None or bar is None:
        raise FileNotFoundError("Missing interlaced.png or barrier.png in output_path.")
    if inter.shape[1] != W_px or inter.shape[0] != H_px or bar.shape[1] != W_px or bar.shape[0] != H_px:
        raise ValueError("Barrier/interlaced dimensions don’t match W_px×H_px.")

    N = int((b_px // s_px) + 1)  # since b_px=(N-1)*s_px -> N = (b/s)+1

    alpha = bar[:, :, 3] if bar.shape[2] == 4 else np.full((H_px, W_px), 255, np.uint8)
    alpha = (alpha >= 128).astype(np.uint8) * 255  # 0 slit, 255 bar
    bar_bgr = np.zeros_like(inter)  # black bars for preview

    def compose(offset):
        if horizontal_motion:
            off = (-offset if reverse_motion else offset) % W_px
            a = np.concatenate([alpha[:, off:], alpha[:, :off]], axis=1)
        else:
            off = (-offset if reverse_motion else offset) % H_px
            a = np.concatenate([alpha[off:, :], alpha[:off, :]], axis=0)
        a3 = a[..., None]
        return np.where(a3 == 0, inter, bar_bgr)

    frames = [compose(k * s_px) for k in range(N)]
    frames = frames * max(1, int(cycles))

    # GIF
    gif_path = os.path.join(output_path, gif_name)
    imageio.mimsave(gif_path, [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames],
                    duration=1.0/max(1,fps), loop=0)

    # MOV
    mov_path = os.path.join(output_path, mov_name)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vw = cv2.VideoWriter(mov_path, fourcc, float(fps), (W_px, H_px))
    if not vw.isOpened():
        raise RuntimeError("Failed to open VideoWriter. Try 'avc1' or 'MJPG'.")
    for f in frames: vw.write(f)
    vw.release()

    print(f"[view] N={N} s_px={s_px} b_px={b_px} W%={W_px % s_px} H%={H_px % s_px}")
    print("Saved:", gif_path, mov_path)
    return gif_path, mov_path

## Scanimation

Create animated images that **come to life when viewed through a striped barrier**.  
The process interlaces frames from a video or image sequence, then overlays them with a matching barrier. When the barrier is moved or the viewer’s perspective changes, the frames are revealed one at a time — producing the illusion of motion.

**Input**
- Video path **or** directory of images  
- Desired image width/height (in units or pixels)  
- *(Optional)* Barrier width, slit width, objects to detect and segment  

**Output**
- **Barrier** (PDF + SVG for printing)  
- **Interlaced image** (PNG/SVG/PDF for printing)  
- **Animated preview** (GIF/MOV) to visualize motion 

In [32]:
# ------------- PARAMETERS -------------
# Edit here

# Path to video or path to a folder containing images 
video_path = None # If None, will use images_path
images_path = './seal/images' # images should be numbered in the order you want them to be animated
output_path = f'{images_path}/scanimated'

# Video options:
start_time = 3 # start time in seconds from which to extract images from the video. if not specified, defaults to 0.
end_time = 4 # end time in seconds until which to extract images from the video. if not specified, defaults to the end of the video.
num_images = 10 # number of images to extract from the video timeframe. if not specified, defaults to 6.

# Sizing options:
image_width = 11 # image width. if not specified, default to the width of the first image in the folder.
image_height = 8.5 #  image height. if not specified, default to the height of the first image in the folder.
image_units="in"
image_size_strict = False # if True image size will be exactly as specified and not be adjusted to make a perfect fit with the barrier. If False image size will be slighty cropped to fit the specified barrier geometry.
barrier_width = 3.0 # width of the moving lines. If None, it will be calculated based on the number of images.
slit_width = None # width of the slits in the interlaced image and in the openings of the barrier. If None, it will be calculated based on the number of images and barrier width.
barrier_units="mm"
barrier_width_strict = True # if True barrier bar width will be exactly as specified and needed the last bar will be cropped to fit the image size. If false barrier bar width will be adjusted (within a certain allowance) so that only full bars and slit openings are included.

# Image options:
objects_to_detect = ['seal'] # if no objects are specified, the entire image will be used. if object is specified, it will be detected in each image and will be overlaid on a white background.
color = 'black' # color to make the image and the the barrier lines. if None, image will not be monocolorized.
invert = False # if True, the color of the object(s) will be white and the background will be the previously specified color.
horizontal_motion = True # True for horizontal motion (vertical lines). False for vertical motion (horizontal lines).
reverse_motion = True # False to view scanimation left to right with horizontal motion and top to bottom with vertical motion. True to reverse

# ------------- RUN -------------
# Leave this as is

if video_path:
    images_path = get_images_from_video(video_path, num_images=num_images, start_time=start_time, end_time=end_time)
# Count how many images there are in images_path 
num_images = count_images(images_path)
# Calculate ideal fit given the specified parameters
image_width, image_height, barrier_width, slit_width = calculate_ideal_fit(images_path, image_width=image_width, image_height=image_height, image_units=image_units, barrier_width=barrier_width, slit_width=slit_width, barrier_units=barrier_units, num_images=num_images, image_size_strict=image_size_strict, barrier_width_strict=barrier_width_strict, dpi=300, suggest_target_slit_mm=1.0, periods_hint_range=(60, 240), tol_rel=0.02)
# Calculate measurements in px for consistency
W_px, H_px, b_px, s_px = measurements_to_px(image_width, image_height, barrier_width, slit_width, image_units=image_units, barrier_units=barrier_units, num_images=num_images, dpi=300)
# Create scanimation barrier
create_scanimation_barrier(output_path, barrier_width=barrier_width, slit_width=slit_width, image_width=image_width, image_height=image_height, image_units=image_units, W_px=W_px, H_px=H_px, b_px=b_px, s_px=s_px, barrier_units=barrier_units, horizontal_motion=horizontal_motion, color=color, svg_out="barrier.svg", pdf_out="barrier.pdf")
# Read and prepare images to be interlaced
processing_path = identify_objects(images_path, objects_to_detect)
processing_path = monocolorize_images(processing_path, color=color, invert=invert)
processing_path = resize_images(processing_path, image_width=W_px, image_height=H_px, units='px', dpi=300, preserve_aspect_ratio=True, crop_to_fit=True)
# Interlace the images into one image
interlace_images(processing_path, output_path, W_px=W_px, H_px=H_px, b_px=b_px, s_px=s_px, horizontal_motion=horizontal_motion, dpi=300)
# Preview scanimation
view_scanimation(output_path, W_px=W_px, H_px=H_px, b_px=b_px, s_px=s_px, horizontal_motion=horizontal_motion, reverse_motion=reverse_motion, fps=12, cycles=2, gif_name="preview.gif", mov_name="preview.mov")



=== Ideal Fit ===
N (num_images): 6
Image size: 11.0551in × 8.5000in
Slit width: 0.6000mm
Barrier width (opaque): 3.0000mm
Period (= N*slit): 3.6000mm
Periods across width: ~78


/var/folders/b1/v2f_7xl50rx0d8ldw7z6kpvw0000gn/T/ipykernel_21351/2068930844.py:218: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(rgba, mode="RGBA")



image 1/1 /Users/macadmin/Desktop/Sara_Laptop/CodingProjects/Scanimation/github/scanimation/examples/seal/images/frame_003.png: 640x352 1 bird, 110.3ms
Speed: 1.4ms preprocess, 110.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 352)

image 1/1 /Users/macadmin/Desktop/Sara_Laptop/CodingProjects/Scanimation/github/scanimation/examples/seal/images/frame_002.png: 640x352 1 bird, 124.6ms
Speed: 2.3ms preprocess, 124.6ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 352)

image 1/1 /Users/macadmin/Desktop/Sara_Laptop/CodingProjects/Scanimation/github/scanimation/examples/seal/images/frame_001.png: 640x352 1 bird, 1 bear, 140.3ms
Speed: 34.2ms preprocess, 140.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 352)

image 1/1 /Users/macadmin/Desktop/Sara_Laptop/CodingProjects/Scanimation/github/scanimation/examples/seal/images/frame_005.png: 640x352 1 bird, 150.1ms
Speed: 1.5ms preprocess, 150.1ms inference, 2.8ms postprocess per image at shape (1, 

('./seal/images/scanimated/preview.gif',
 './seal/images/scanimated/preview.mov')

## Lamp Template

Generate a cylindrical lamp template where the perimeter is divided into alternating **barriers** and **slits**.  
Supports **circle**, **square**, and **heart** outlines. Outputs both **SVG** and **PDF** (with an optional extra page of evenly spaced lines for printing).

**Input**
- Barrier width (e.g. `3.0 mm`)  
- Slit width (optional, auto-calculated if `None`)  
- Desired perimeter distance (in chosen units)  
- Shape (`"circle"`, `"square"`, or `"heart"`)  
- Units for barrier/slit vs perimeter (`barrier_units`, `perimeter_units`)  
- *(Optional)* Flags like `extend_lines=True` to draw extended center/outward lines  

**Output**
- `lamp_template.svg` — main lamp outline with ticks, dots, and specs  
- `lamp_template.pdf` — multi-page PDF (page 1 template, page 2 vertical lines)  
- `lamp_template_lines.svg` — extra page of evenly spaced lines (for barrier strips)  


In [33]:
def create_lamp_template(
    barrier_width,
    slit_width,
    perimeter_distance,
    barrier_units="mm",       # units for barrier/slit/ticks/dots/stroke and canvas
    perimeter_units="mm",     # units for the perimeter length
    shape="circle",           # "circle", "square", or "heart"
    output_path=".",
    svg_out="lamp_template.svg",
    pdf_out="lamp_template.pdf",
    outline_stroke=0.3,       # in barrier_units
    tick_len=None,            # default: 0.8 * barrier_width (in barrier_units)
    inset=None,               # default: 0.6 * barrier_width (in barrier_units)
    dot_radius=None,          # default: 0.18 * barrier_width (in barrier_units)
    samples_heart=1200,       # sampling resolution for the heart path
    text_size=None,           # center text size (in barrier_units); auto if None
    print_specs=True,
    extend_lines=False        # extend ticks to center and outward; outward pair kept parallel
):
    """
    Creates an outline (circle/square/heart) whose perimeter equals `perimeter_distance`
    (interpreted in `perimeter_units`). All other geometry (bar widths, ticks, dots, stroke,
    canvas units) is in `barrier_units`.

    Output:
      - <svg_out>                : main lamp template (SVG)
      - <stem>_lines.svg         : lines page (SVG) with vertical bars on 8.5x11 portrait
      - <pdf_out>                : two-page PDF (page1 template, page2 lines)

    Specs always print with 3 decimals. If extend_lines=True:
      - inward lines go to the center; outward lines go beyond the perimeter,
        start/end outward segments are parallel (outward normal at bar-mid).
      - a red dot marks the center.
    """
    import os
    import math
    from math import pi, cos, sin

    # ---------- Output paths ----------
    os.makedirs(output_path, exist_ok=True)
    svg_out = os.path.join(output_path, svg_out)
    pdf_out = os.path.join(output_path, pdf_out)
    lines_svg_path = os.path.splitext(svg_out)[0] + "_lines.svg"

    # ---------- Unit helpers ----------
    def _mm_per_unit(u: str) -> float:
        u = (u or "").lower()
        if u == "mm": return 1.0
        if u == "cm": return 10.0
        if u in ("in", "inch", "inches"): return 25.4
        if u in ("px", "pixel", "pixels"): return 25.4 / 96.0  # 96 px/in
        raise ValueError(f"Unsupported unit for mm conversion: {u}")

    def _pt_per_unit(u: str) -> float:
        u = (u or "").lower()
        if u in ("pt", "pts", "point", "points"): return 1.0
        if u in ("in", "inch", "inches"): return 72.0
        if u == "mm": return 72.0 / 25.4
        if u == "cm": return 72.0 / 2.54
        if u in ("px", "pixel", "pixels"): return 72.0 / 96.0
        raise ValueError(f"Unsupported unit for pt conversion: {u}")

    def _inch_per_unit(u: str) -> float:
        u = (u or "").lower()
        if u in ("in", "inch", "inches"): return 1.0
        if u == "mm": return 1.0 / 25.4
        if u == "cm": return 1.0 / 2.54
        if u in ("pt", "pts", "point", "points"): return 1.0 / 72.0
        if u in ("px", "pixel", "pixels"): return 1.0 / 96.0
        raise ValueError(f"Unsupported unit for inch conversion: {u}")

    def _fmt(n: float) -> str:
        # geometry numbers in streams
        return f"{n:.6f}".rstrip("0").rstrip(".")

    # ---------- PDF text helpers ----------
    def _pdf_helv_char_width_1000u(ch: str) -> int:
        table = {
            " ":278, "-":333, ".":278, ",":278, ":":278, "/":278, "|":280,
            "0":556,"1":556,"2":556,"3":556,"4":556,"5":556,"6":556,"7":556,"8":556,"9":556,
            "A":667,"B":667,"C":722,"D":722,"E":667,"F":611,"G":778,"H":722,"I":278,"J":500,
            "K":667,"L":556,"M":833,"N":722,"O":778,"P":667,"Q":778,"R":722,"S":667,"T":611,
            "U":722,"V":667,"W":944,"X":667,"Y":667,"Z":611,
            "a":556,"b":556,"c":500,"d":556,"e":556,"f":278,"g":556,"h":556,"i":222,"j":222,
            "k":500,"l":222,"m":833,"n":556,"o":556,"p":556,"q":556,"r":333,"s":500,"t":278,
            "u":556,"v":500,"w":722,"x":500,"y":500,"z":500,
        }
        return table.get(ch, 600)

    def _pdf_helv_text_width_pts(s: str, font_size_pt: float) -> float:
        total_1000 = sum(_pdf_helv_char_width_1000u(ch) for ch in s)
        return (total_1000 / 1000.0) * font_size_pt

    def _esc_pdf_string(s: str) -> str:
        return s.replace("\\", "\\\\").replace("(", "\\(").replace(")", "\\)")

    # ---------- Inputs normalization ----------
    barrier_width       = float(barrier_width)
    slit_width          = float(slit_width)
    perimeter_distance  = float(perimeter_distance)

    if barrier_width <= 0 or slit_width < 0 or perimeter_distance <= 0:
        raise ValueError("barrier_width>0, slit_width>=0, perimeter_distance>0 required")

    # Convert perimeter length into *barrier_units* for geometry construction
    mm_per_bar = _mm_per_unit(barrier_units)
    mm_per_per = _mm_per_unit(perimeter_units)
    per_to_bar = mm_per_per / mm_per_bar
    perimeter_bar = perimeter_distance * per_to_bar  # perimeter in barrier_units

    period = barrier_width + slit_width
    if tick_len is None:
        tick_len = 0.8 * barrier_width
    if inset is None:
        inset = 0.6 * barrier_width
    if dot_radius is None:
        dot_radius = 0.18 * barrier_width

    images_est = max(1, int(round((barrier_width + slit_width) / max(1e-9, slit_width))))

    # Number of periods around the perimeter
    periods_exact = perimeter_bar / period            # in periods (can be non-integer)
    periods_str   = f"{periods_exact:.3f}"            # 3 decimals for display

    # ---------- Polyline helpers ----------
    def _polyline_length(pts):
        total = 0.0
        for i in range(1, len(pts)):
            x0,y0 = pts[i-1]; x1,y1 = pts[i]
            total += math.hypot(x1-x0, y1-y0)
        return total

    def _resample_polyline_by_arc(pts, s_vals):
        cum = [0.0]
        for i in range(1, len(pts)):
            cum.append(cum[-1] + math.hypot(pts[i][0]-pts[i-1][0], pts[i][1]-pts[i-1][1]))
        total = cum[-1]
        out = []
        for s in s_vals:
            s = s % total
            lo, hi = 0, len(cum)-1
            while lo < hi:
                mid = (lo+hi)//2
                if cum[mid] <= s < cum[mid+1]:
                    lo = mid
                    break
                elif cum[mid] < s:
                    lo = mid+1
                else:
                    hi = mid
            i = min(lo, len(pts)-2)
            seg_len = cum[i+1]-cum[i]
            t = 0.0 if seg_len <= 1e-12 else (s - cum[i]) / seg_len
            x = pts[i][0] + t*(pts[i+1][0]-pts[i][0])
            y = pts[i][1] + t*(pts[i+1][1]-pts[i][1])
            out.append((x,y,i,t))
        return out, total

    def _estimate_tangent_normal(pts, i, t):
        x0,y0 = pts[i]; x1,y1 = pts[i+1]
        tx, ty = x1-x0, y1-y0
        L = math.hypot(tx,ty)
        if L == 0: return (1.0,0.0),(0.0,1.0)
        tx/=L; ty/=L
        return (tx,ty),(-ty,tx)

    def _point_plus(vec, x, y, d):
        vx, vy = vec
        return x + d*vx, y + d*vy

    # ---------- Build outline in *barrier_units* ----------
    pts = []
    s_lower = shape.lower()
    if s_lower == "circle":
        R = perimeter_bar / (2*pi)
        cx = cy = R
        steps = 720
        for k in range(steps+1):
            th = 2*pi*k/steps
            pts.append((cx + R*cos(th), cy + R*sin(th)))
    elif s_lower == "square":
        L = perimeter_bar / 4.0
        pts = [(0,0),(L,0),(L,L),(0,L),(0,0)]
    elif s_lower == "heart":
        samples_heart = int(max(400, samples_heart))
        t_vals = [2*pi*k/samples_heart for k in range(samples_heart+1)]
        raw = [(16*(math.sin(t)**3),
                13*math.cos(t) - 5*math.cos(2*t) - 2*math.cos(3*t) - math.cos(4*t))
               for t in t_vals]
        raw_len = _polyline_length(raw)
        if raw_len <= 0:
            raise RuntimeError("Heart perimeter computation failed.")
        scale = perimeter_bar / raw_len
        pts = [(x*scale, y*scale) for (x,y) in raw]
        minx = min(p[0] for p in pts); miny = min(p[1] for p in pts)
        pts = [(x-minx, y-miny) for (x,y) in pts]
    else:
        raise ValueError("shape must be 'circle', 'square', or 'heart'")

    # Canvas extents and margin (in barrier_units)
    minx = min(p[0] for p in pts); maxx = max(p[0] for p in pts)
    miny = min(p[1] for p in pts); maxy = max(p[1] for p in pts)
    width = maxx-minx; height = maxy-miny
    margin = max(period, outline_stroke*10, tick_len + inset + max(dot_radius, 2.0)) + 3.0

    # ---------- Bar positions along outline (barrier_units) ----------
    s_vals=[]; s = slit_width  # start with a slit at s=0
    perimeter_len = perimeter_bar
    while s < perimeter_len - 1e-9:
        s0=s; s1=min(s+barrier_width, perimeter_len); sm=0.5*(s0+s1)
        s_vals.extend([s0,s1,sm])
        s += period

    samples, total_len = _resample_polyline_by_arc(pts, s_vals)
    centroid_x = sum(p[0] for p in pts[:-1]) / max(1,(len(pts)-1))
    centroid_y = sum(p[1] for p in pts[:-1]) / max(1,(len(pts)-1))
    center = (centroid_x, centroid_y)

    def inward(nx,ny,x,y):
        # choose normal pointing inward (towards centroid)
        vx,vy = centroid_x-x, centroid_y-y
        return (nx,ny) if nx*vx + ny*vy > 0 else (-nx,-ny)

    # ---------- Build SVG elements (main page) ----------
    def shift(p): return (p[0]+margin, p[1]+margin)

    outline_points_str = " ".join(f"{p[0]+margin:.6f},{p[1]+margin:.6f}" for p in pts)
    svg_elems = [f'<polyline points="{outline_points_str}" fill="none" stroke="#000" stroke-width="{outline_stroke:.6f}"/>' ]

    ticks=[]; dots=[]
    center_dot_pos = shift(center)  # for SVG red dot

    for k in range(0, len(samples), 3):
        (x0,y0,i0,t0) = samples[k]     # bar start
        (x1,y1,i1,t1) = samples[k+1]   # bar end
        (xm,ym,im,tm) = samples[k+2]   # bar mid

        (tx0,ty0),(nx0,ny0) = _estimate_tangent_normal(pts,i0,t0)
        (tx1,ty1),(nx1,ny1) = _estimate_tangent_normal(pts,i1,t1)
        (txm,tym),(nxm,nym) = _estimate_tangent_normal(pts,im,tm)

        nx0,ny0 = inward(nx0,ny0,x0,y0)
        nx1,ny1 = inward(nx1,ny1,x1,y1)
        nxm,nym = inward(nxm,nym,xm,ym)

        if extend_lines:
            # Outward direction shared by start & end: outward normal at bar mid
            outward_dir = (-nxm, -nym)  # already unit
            out_len = margin

            # Inward segments: point -> center
            ticks.append(((x0,y0), center))
            ticks.append(((x1,y1), center))

            # Outward segments: parallel for both ends
            p0_out = _point_plus(outward_dir, x0, y0, out_len)
            p1_out = _point_plus(outward_dir, x1, y1, out_len)
            ticks.append(((x0,y0), p0_out))
            ticks.append(((x1,y1), p1_out))
        else:
            # Short, fully-inside ticks (original behavior)
            tip0  = _point_plus((nx0,ny0), x0,y0, outline_stroke/2)
            base0 = _point_plus((nx0,ny0), x0,y0, outline_stroke/2 + tick_len)
            tip1  = _point_plus((nx1,ny1), x1,y1, outline_stroke/2)
            base1 = _point_plus((nx1,ny1), x1,y1, outline_stroke/2 + tick_len)
            ticks.append((base0,tip0)); ticks.append((base1,tip1))

        # Dot at bar mid, inset inward
        dot_c = _point_plus((nxm,nym), xm,ym, max(0.0, outline_stroke/2 + inset))
        dots.append(dot_c)

    # SVG ticks and dots
    for (p0,p1) in ticks:
        x1,y1 = shift(p0); x2,y2 = shift(p1)
        svg_elems.append(
            f'<line x1="{x1:.6f}" y1="{y1:.6f}" x2="{x2:.6f}" y2="{y2:.6f}" '
            f'stroke="#000" stroke-width="{outline_stroke:.6f}" stroke-linecap="round"/>'
        )
    for (xd,yd) in dots:
        x,y = shift((xd,yd))
        svg_elems.append(f'<circle cx="{x:.6f}" cy="{y:.6f}" r="{dot_radius:.6f}" fill="#000"/>')

    # Red center dot (only when extend_lines=True)
    if extend_lines:
        svg_elems.append(
            f'<circle cx="{center_dot_pos[0]:.6f}" cy="{center_dot_pos[1]:.6f}" '
            f'r="{max(outline_stroke*2.0, dot_radius*0.6):.6f}" fill="#d00"/>'
        )

    # Centered specs text (3 decimals always)
    svg_W = width + 2*margin; svg_H = height + 2*margin
    cx_canvas = svg_W/2.0; cy_canvas = svg_H/2.0
    if text_size is None:
        text_size = max(2.5, min(svg_W, svg_H) * 0.03)

    if print_specs:
        line1 = f"Shape: {shape.capitalize()}  |  Perimeter: {perimeter_distance:.3f}{perimeter_units}"
        line2 = (
            f"Barrier: {barrier_width:.3f}{barrier_units}  |  "
            f"Slit: {slit_width:.3f}{barrier_units}  |  "
            f"Period: {period:.3f}{barrier_units}"
        )
        line3 = f"N images: {images_est}  |  Periods: {periods_exact:.3f}"

        svg_elems.append(
            f'<text x="{cx_canvas:.6f}" y="{cy_canvas - 1.2*text_size:.6f}" '
            f'font-family="Arial, Helvetica, sans-serif" font-size="{text_size:.6f}" '
            f'text-anchor="middle" fill="#000">{line1}</text>')
        svg_elems.append(
            f'<text x="{cx_canvas:.6f}" y="{cy_canvas:.6f}" '
            f'font-family="Arial, Helvetica, sans-serif" font-size="{text_size:.6f}" '
            f'text-anchor="middle" fill="#000">{line2}</text>')
        svg_elems.append(
            f'<text x="{cx_canvas:.6f}" y="{cy_canvas + 1.2*text_size:.6f}" '
            f'font-family="Arial, Helvetica, sans-serif" font-size="{text_size:.6f}" '
            f'text-anchor="middle" fill="#000">{line3}</text>')

    # ---------- Write main SVG ----------
    svg_text = f'''<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<svg xmlns="http://www.w3.org/2000/svg"
     width="{svg_W:.6f}{barrier_units}" height="{svg_H:.6f}{barrier_units}"
     viewBox="0 0 {svg_W:.6f} {svg_H:.6f}">
  <title>Lamp Template ({shape.capitalize()}, Ticks + Specs)</title>
  <desc>Perimeter={perimeter_distance:.3f}{perimeter_units}; bar={barrier_width:.3f}{barrier_units}, slit={slit_width:.3f}{barrier_units}</desc>
  {''.join(svg_elems)}
</svg>
'''
    with open(svg_out, "w", encoding="utf-8") as f:
        f.write(svg_text)

    # ---------- LINES PAGE (SVG) — vertical lines on 8.5x11 portrait ----------
    page_w_in, page_h_in = 8.5, 11.0  # portrait
    spacing_in = barrier_width * _inch_per_unit(barrier_units)  # convert barrier_units -> inches

    svg_lines = [ '<?xml version="1.0" encoding="UTF-8" standalone="no"?>',
                  f'<svg xmlns="http://www.w3.org/2000/svg" width="{page_w_in}in" height="{page_h_in}in" viewBox="0 0 {page_w_in} {page_h_in}">',
                  '<title>Lamp Template — Lines Page (Vertical)</title>',
                  '<g stroke="#000" stroke-width="0.003in" fill="none">' ]

    x = 0.0
    while x <= page_w_in + 1e-9:
        svg_lines.append(f'<line x1="{x:.6f}" y1="0" x2="{x:.6f}" y2="{page_h_in:.6f}"/>')
        x += spacing_in

    svg_lines.append('</g></svg>')
    with open(lines_svg_path, "w", encoding="utf-8") as f:
        f.write("\n".join(svg_lines))

    # ---------- Minimal Vector PDF (2 pages: main + lines) ----------
    U = _pt_per_unit(barrier_units)  # points per barrier unit
    pts_pt   = [((x+margin)*U, (y+margin)*U) for (x,y) in pts]
    ticks_pt = [(((p0[0]+margin)*U,(p0[1]+margin)*U), ((p1[0]+margin)*U,(p1[1]+margin)*U)) for (p0,p1) in ticks]
    dots_pt  = [((x+margin)*U,(y+margin)*U) for (x,y) in dots]
    sw_pt    = outline_stroke * U
    dot_pt   = dot_radius * U

    svg_W_pt, svg_H_pt = svg_W * U, svg_H * U
    cx_pt, cy_pt = (cx_canvas * U), (cy_canvas * U)
    font_size_pt = (text_size or max(2.5, min(svg_W, svg_H)*0.03)) * U

    if print_specs:
        line1 = f"Shape: {shape.capitalize()}  |  Perimeter: {perimeter_distance:.3f}{perimeter_units}"
        line2 = (
            f"Barrier: {barrier_width:.3f}{barrier_units}  |  "
            f"Slit: {slit_width:.3f}{barrier_units}  |  "
            f"Period: {period:.3f}{barrier_units}"
        )
        line3 = f"N images: {images_est}  |  Periods: {periods_exact:.3f}"

    # center (pt)
    center_pt = ((center[0] + margin) * U, (center[1] + margin) * U)
    center_dot_r_pt = max(outline_stroke*2.0, dot_radius*0.6) * U

    # ---- Page 1 content (main template) ----
    stream1 = []
    # Outline (black)
    stream1 += ["0 0 0 RG", f"{_fmt(max(0.2*_pt_per_unit('pt'), sw_pt))} w"]
    if pts_pt:
        x0,y0 = pts_pt[0]; stream1.append(f"{_fmt(x0)} {_fmt(y0)} m")
        for (x,y) in pts_pt[1:]:
            stream1.append(f"{_fmt(x)} {_fmt(y)} l")
        stream1.append("S")
    # Ticks (black)
    stream1 += ["0 0 0 RG", f"{_fmt(sw_pt)} w"]
    for (p0,p1) in ticks_pt:
        (x1,y1),(x2,y2) = p0,p1
        stream1.append(f"{_fmt(x1)} {_fmt(y1)} m {_fmt(x2)} {_fmt(y2)} l S")
    # Dots at bar mids (black squares)
    for (x,y) in dots_pt:
        r = dot_pt
        stream1 += ["0 0 0 rg", f"{_fmt(x - r)} {_fmt(y - r)} {_fmt(2*r)} {_fmt(2*r)} re f"]

    # Red center dot if extend_lines
    if extend_lines:
        r = center_dot_r_pt
        cxr, cyr = center_pt
        stream1 += ["1 0 0 rg", f"{_fmt(cxr - r)} {_fmt(cyr - r)} {_fmt(2*r)} {_fmt(2*r)} re f"]
        stream1 += ["0 0 0 rg", "0 0 0 RG"]

    # Center text
    stream1 += ["BT", f"/F1 {font_size_pt:.4f} Tf"]
    if print_specs:
        def _centered_Tj(line: str, cx: float, y: float):
            w = _pdf_helv_text_width_pts(line, font_size_pt)
            x_left = cx - w / 2.0
            stream1.append(f"1 0 0 1 {_fmt(x_left)} {_fmt(y)} Tm ({_esc_pdf_string(line)}) Tj")
        _centered_Tj(line1, cx_pt, cy_pt + 1.2*font_size_pt)
        _centered_Tj(line2, cx_pt, cy_pt)
        _centered_Tj(line3, cx_pt, cy_pt - 1.2*font_size_pt)
    stream1 += ["ET"]

    content1 = "\n".join(stream1).encode("ascii")

    # ---- Page 2 content (vertical lines on 8.5x11 portrait) ----
    page_w_in, page_h_in = 8.5, 11.0
    pageWpt, pageHpt = page_w_in * 72.0, page_h_in * 72.0
    spacing_pt = barrier_width * _pt_per_unit(barrier_units)  # spacing in points from barrier_width

    lw2 = max(0.3, outline_stroke * _pt_per_unit(barrier_units))  # stroke width for lines
    stream2 = [ "0 0 0 RG", f"{_fmt(lw2)} w" ]
    x = 0.0
    while x <= pageWpt + 0.5:
        stream2.append(f"{_fmt(x)} 0 m {_fmt(x)} {_fmt(pageHpt)} l S")
        x += spacing_pt

    content2 = "\n".join(stream2).encode("ascii")

    # ---- Build 2-page PDF ----
    header = "%PDF-1.4\n%\xE2\xE3\xCF\xD3\n"
    obj_strs = []

    def add(s: str):
        obj_strs.append(s)

    # 1 Catalog
    # 2 Pages (refs 3 and 6)
    # 3 Page (main) -> contents 4, font 5
    # 4 Contents (main)
    # 5 Font
    # 6 Page (lines) -> contents 7
    # 7 Contents (lines)

    add("1 0 obj\n<< /Type /Catalog /Pages 2 0 R >>\nendobj\n")
    add("2 0 obj\n<< /Type /Pages /Count 2 /Kids [3 0 R 6 0 R] >>\nendobj\n")

    add(
        f"3 0 obj\n<< /Type /Page /Parent 2 0 R /MediaBox [0 0 {_fmt(svg_W_pt)} {_fmt(svg_H_pt)}] "
        f"/Resources << /Font << /F1 5 0 R >> >> /Contents 4 0 R >>\nendobj\n"
    )
    add(
        f"4 0 obj\n<< /Length {len(content1)} >>\nstream\n{content1.decode('ascii')}\nendstream\nendobj\n"
    )
    add("5 0 obj\n<< /Type /Font /Subtype /Type1 /BaseFont /Helvetica >>\nendobj\n")

    add(
        f"6 0 obj\n<< /Type /Page /Parent 2 0 R /MediaBox [0 0 {_fmt(pageWpt)} {_fmt(pageHpt)}] "
        f"/Contents 7 0 R >>\nendobj\n"
    )
    add(
        f"7 0 obj\n<< /Length {len(content2)} >>\nstream\n{content2.decode('ascii')}\nendstream\nendobj\n"
    )

    with open(pdf_out, "wb") as f:
        # write header
        f.write(header.encode("latin-1"))
        # write objects, collecting offsets
        offsets = []
        for s in obj_strs:
            offsets.append(f.tell())
            f.write(s.encode("latin-1"))
        startxref = f.tell()
        # xref
        f.write(f"xref\n0 {len(offsets)+1}\n".encode("latin-1"))
        f.write(b"0000000000 65535 f \n")
        for off in offsets:
            f.write(f"{off:010d} 00000 n \n".encode("latin-1"))
        # trailer
        f.write(f"trailer\n<< /Size {len(offsets)+1} /Root 1 0 R >>\nstartxref\n{startxref}\n%%EOF\n".encode("latin-1"))

    return svg_out, pdf_out, lines_svg_path

In [34]:
# CREATE LAMP TEMPLATE
# If you would like to create a lamp for your interlaced image, create a template using the corresponding barrier geometry 

num_images = num_images
barrier_width = barrier_width
slit_width = slit_width
barrier_units = barrier_units
perimeter_distance = image_width
perimeter_units = image_units
shape = "circle"
output_path = output_path
svg_out = f"lamp_template_{shape}_N_{num_images}_W_{perimeter_distance:.2f}{perimeter_units}.svg"
pdf_out = f"lamp_template_{shape}_N_{num_images}_W_{perimeter_distance:.2f}{perimeter_units}.pdf"
print_specs = True,
extend_lines = False

create_lamp_template(
    barrier_width=barrier_width,
    slit_width=slit_width,               
    perimeter_distance=perimeter_distance,       
    barrier_units=barrier_units,
    perimeter_units = perimeter_units,               
    shape=shape,
    output_path=output_path,
    svg_out=svg_out,
    pdf_out=pdf_out,
    print_specs=print_specs,
    extend_lines=extend_lines
)

('./seal/images/scanimated/lamp_template_circle_N_6_W_11.06in.svg',
 './seal/images/scanimated/lamp_template_circle_N_6_W_11.06in.pdf',
 './seal/images/scanimated/lamp_template_circle_N_6_W_11.06in_lines.svg')